# Iris Neural Network

In [17]:
# Reference links:
#    Adapted from: https://github.com/emerging-technologies/keras-iris
#    Data set pulled from: https://gist.github.com/curran/a08a1080b88344b0c8a7

# imports and preliminaries
import csv
import numpy as np
import keras as kr

# Load the Iris dataset.
iris = list(csv.reader(open('iris-data-set.csv')))[1:]

Now that we have the data set loaded we can extrapolate the data we need into appropriate data sets in preparation for training and testing our Model

In [21]:
# The inputs are four floats: sepal length, sepal width, petal length, petal width.
inputs  = np.array(iris)[:,:4].astype(np.float)

In [19]:
# Outputs are initially individual strings: setosa, versicolor or virginica.
outputs = np.array(iris)[:,4]

# Convert the output strings to ints.
outputs_vals, outputs_ints = np.unique(outputs, return_inverse=True)

# Encode the category integers as binary categorical vairables.
outputs_cats = kr.utils.to_categorical(outputs_ints)

We can now randomly split the data into two sets: One for training and one for evaluation

In [26]:
# Split the input and output data sets into training and test subsets.
inds = np.random.permutation(len(inputs))
train_inds, test_inds = np.array_split(inds, 2)
inputs_train, outputs_train = inputs[train_inds], outputs_cats[train_inds]
inputs_test,  outputs_test  = inputs[test_inds],  outputs_cats[test_inds]

We can now create a model

- Notes here about sigmoid

- Notes here about softmax

In [32]:
# Create a neural network.
model = kr.models.Sequential()

# Add an initial layer with 4 input nodes, and a hidden layer with 16 nodes.
model.add(kr.layers.Dense(16, input_shape=(4,)))
# Apply the sigmoid activation function to that layer.
model.add(kr.layers.Activation("sigmoid"))
# Add another layer, connected to the layer with 16 nodes, containing three output nodes.
model.add(kr.layers.Dense(3))
# Use the softmax activation function there.
model.add(kr.layers.Activation("softmax"))


In [34]:
# Display our Model using the summary function
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


## Configure the Model for training and fit the training data

In [35]:
# Configure the model for training.
# Uses the adam optimizer and categorical cross entropy as the loss function.
# Add in some extra metrics - accuracy being the only one.
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Fit the model using our training data.
model.fit(inputs_train, outputs_train, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
75/75 [==============================] - 1s 7ms/step - loss: 1.2385 - acc: 0.3600
Epoch 2/100
75/75 [==============================] - 0s 3ms/step - loss: 1.1447 - acc: 0.3600
Epoch 3/100
75/75 [==============================] - 0s 3ms/step - loss: 1.0934 - acc: 0.3467
Epoch 4/100
75/75 [==============================] - 0s 3ms/step - loss: 1.0541 - acc: 0.3467
Epoch 5/100
75/75 [==============================] - 0s 3ms/step - loss: 1.0108 - acc: 0.3600
Epoch 6/100
75/75 [==============================] - 0s 3ms/step - loss: 0.9692 - acc: 0.4933
Epoch 7/100
75/75 [==============================] - 0s 4ms/step - loss: 0.9259 - acc: 0.6400
Epoch 8/100
75/75 [==============================] - 0s 4ms/step - loss: 0.8789 - acc: 0.6400
Epoch 9/100
75/75 [==============================] - 0s 3ms/step - loss: 0.8395 - acc: 0.6400
Epoch 10/100
75/75 [==============================] - 0s 4ms/step - loss: 0.7969 - acc: 0.6933
Epoch 11/100
75/75 [==============================] - 0s 3m

75/75 [==============================] - 0s 3ms/step - loss: 0.1436 - acc: 0.9733
Epoch 88/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1418 - acc: 0.9733
Epoch 89/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1397 - acc: 0.9733
Epoch 90/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1373 - acc: 0.9733
Epoch 91/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1359 - acc: 0.9733
Epoch 92/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1363 - acc: 0.9733
Epoch 93/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1331 - acc: 0.9733
Epoch 94/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1304 - acc: 0.9733
Epoch 95/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1321 - acc: 0.9733
Epoch 96/100
75/75 [==============================] - 0s 3ms/step - loss: 0.1261 - acc: 0.9733
Epoch 97/100
75/75 [==============================] - 0s 3ms/st

## Evaluate the Loss and Accuracy of the Model

In [36]:
# Evaluate the model using the test data set.
loss, accuracy = model.evaluate(inputs_test, outputs_test, verbose=1)

# Output the accuracy of the model.
print("\n\nLoss: %6.4f\tAccuracy: %6.4f" % (loss, accuracy))

75/75 [==============================] - 0s 228us/step


Loss: 0.1291	Accuracy: 0.9600
